In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image, ImageTk

In [5]:
model = load_model('mnist-3-5.h5')  # mnist-3-2.h5 also working fine

def predict_digit(img):
    #resize image to 75x75 pixels
    img = img.resize((75, 75))
    
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    
#     print('Image to be predicted...')
#     plt.imshow(img, cmap = 'gray')
#     plt.show()
    
    #reshaping to support our model input and normalizing
    img = img.reshape(1,75,75,1).astype(float)
    img = img/255.0
    

    #predicting the class
    res = model.predict([img])
    digits_res = res[0].flatten()
    digit = np.argmax(digits_res)
    acc = digits_res[digit]
    box = res[1].flatten()

    return digit, acc, box


class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.title('Handwritten Digit Recognition and Localization')
        self.canvas1 = tk.Canvas(self, width=500, height=500, bg = 'black', cursor="cross")
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
    # Grid structure
        self.canvas1.grid(row=0, column=0, pady=0, sticky=W, rowspan = 2, columnspan = 2)
        self.classify_btn.grid(row=2, column=1, pady=2, padx = 2)
        self.button_clear.grid(row=2, column=0, pady=2)
        
        self.canvas1.bind("<B1-Motion>", self.draw_lines) # event handler for mouse events

    def clear_all(self):
        self.canvas1.delete("all")
        
    def classify_handwriting(self):
        # code to convert drawing on canvas to an image
        HWND = self.canvas1.winfo_id()        # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        self.im = ImageGrab.grab(rect)
    
        # predict what the image is...
        digit, acc, box = predict_digit(self.im)
        
        w, h = self.im.size
        
        box1 = box[0] * h   # height (y_min)
        box2 = box[1] * w   # width (x_min)
        box3 = box[2] * h   # height (y_max)
        box4 = box[3] * w   # width (x_max)

        self.img = ImageTk.PhotoImage(self.im)
        self.canvas1.create_image((w/2, h/2), image=self.img, anchor = "center")
        self.canvas1.create_rectangle(box2, box1, box4, box3, outline = 'red')
        string = str(digit)+ ',' + str(int(acc * 100)) + '%'
        
        if box1<=20:
            self.canvas1.create_text(box2+35, box3 +15, fill = 'red', text = string, 
                                 font="Times 20 italic bold")
        elif box2<=20:
            self.canvas1.create_text(box4-15, box1-15, fill = 'red', text = string, 
                                 font="Times 20 italic bold")
        else:    
            self.canvas1.create_text(box2+35, box1-15, fill = 'red', text = string, 
                                 font="Times 20 italic bold")

    def draw_lines(self, event):
        # draw on the canvas
        self.x = event.x
        self.y = event.y
        r= 6                              # control the width of strokes
        self.canvas1.create_oval(self.x+r, self.y+r, self.x - r, self.y - r, fill='white',outline = 'white')
       
app = App()
mainloop()
